<a href="https://colab.research.google.com/github/kahram-y/first-repository/blob/master/DeepLearning/lyricsgnrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM 기반 가사(lyrics) 생성기 만들기
- 해석 불가능한 문장(의미 없는 반복·단어 폭주) 문장을 개선하기 위한 시도:
    - 데이터 **전처리 개선**: 특수문자/숫자/구두점 삭제를 줄임
    - Sentence 시작 토큰 **< start >** **와** **< end > 토큰 추가**
    - 장기 문맥을 배울 수 없는 문제 개선: **max_len**을 15 (모든 문장을 15 토큰으로 패딩) → **30으로 증가**
    - Argmax (가장 확률 높은 단어만 선택) 제거 → **Temperature** (0.7~0.9 구간은 문장 자연성에 매우 효과적) + **Top-k 샘플링** 적용
- 생성 문장 퀄리티를 최대한 자연스럽게 개선하기 위한 시도:
    - **Attention** (LSTM 기반 Seq2Seq에서 가장 큰 성능 향상 요소)
    - **Bidirectional** (양방향 문맥 활용)
    - **Layer Normalization** + **Dropout** (학습 안정 + 과적합 완화)  *(적용 전)*
    - **Beam Search** 사용: Sampling에 비해 랜덤성 낮고, 가사가 더 논리적,문법적이 됨. Sampling과 Beam Search를 둘 다 포함시켜서 생성 품질을 비교하고 상황에 따라 선택할 수 있도록 했습니다.

# 학습 소요시간
아래 모델 기준 학습에 5 epoch에 30분정도 소요되었습니다

# 텍스트 생성모델의 validaion loss
val_loss: 1.2769

In [1]:
from google.colab import files
uploaded = files.upload()

!ls /content

Saving lyrics.zip to lyrics.zip
lyrics.zip  sample_data


In [2]:
# 데이터 load 및 압축 해제

import os
import zipfile

# 데이터 루트 경로 - 로컬 파일 업로드해서 /content 에 저장한 경우
data_path = '/content'
lyrics_path = os.path.join(data_path, 'lyrics_data')

# 압축 파일 목록 (파일 이름도 실제 파일명으로 수정)
zip_files = {'lyrics.zip': 'lyrics'}

# 업로드 폴더 생성
os.makedirs(lyrics_path, exist_ok=True)

for zip_name, folder_name in zip_files.items():
    zip_path = os.path.join(data_path, zip_name)        # 업로드한 zip 파일 경로
    extract_dir = os.path.join(lyrics_path, folder_name)

    os.makedirs(extract_dir, exist_ok=True)

    print(f"🔍 압축 해제 중: {zip_name} → {extract_dir}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("✅ 모든 압축 해제 완료!")

🔍 압축 해제 중: lyrics.zip → /content/lyrics_data/lyrics
✅ 모든 압축 해제 완료!


In [3]:
import glob #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import os

lyrics_file_path = '/content/lyrics_data/lyrics/lyrics/*.txt'

txt_list = glob.glob(lyrics_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

In [4]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)"]


In [5]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    # 구두점 앞뒤 공백
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence) # 중복 공백 제거

    # <start> <end> 추가
    sentence = "<start> " + sentence + " <end>"
    return sentence

corpus = [preprocess_sentence(s) for s in raw_corpus]

In [6]:
import numpy as np
import tensorflow as tf

# 토크나이저 함수로 Tensor 변환

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,
    filters='',          # 필터 제거 → 의미 보존
    oov_token="<unk>"
)
tokenizer.fit_on_texts(corpus)

tensor = tokenizer.texts_to_sequences(corpus)
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=30)

print("Tensor example:", tensor[0])

Tensor example: [   2  131    4    6   52   91   50  407   16    7    8   62    9  450
   19 1099    3    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [7]:
from sklearn.model_selection import train_test_split

enc_inputs = tensor[:, :-1]
dec_targets = tensor[:, 1:]

# 20%를 평가 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(
    enc_inputs,
    dec_targets,
    test_size=0.2,
    random_state=42,  # 재현성 확보 위해 시드 고정 (선택 사항)
    shuffle=True      # 데이터 섞기
)

In [8]:
enc_train

array([[   2, 2221,    4, ...,    0,    0,    0],
       [   2,   19, 1202, ...,    0,    0,    0],
       [   2,    3,    0, ...,    0,    0,    0],
       ...,
       [   2,  459,   21, ...,    0,    0,    0],
       [   2,    8,   90, ...,    0,    0,    0],
       [   2,   37,   22, ...,    0,    0,    0]], dtype=int32)

In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 64
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 29), dtype=tf.int32, name=None), TensorSpec(shape=(64, 29), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(64, 29), dtype=tf.int32, name=None), TensorSpec(shape=(64, 29), dtype=tf.int32, name=None))>


In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [11]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다.
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
history = model.fit(
    dataset,
    validation_data=val_dataset,
    epochs=5
)

Epoch 1/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 394s 165ms/step - loss: 1.8570 - val_loss: 1.4895
Epoch 2/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 389s 166ms/step - loss: 1.4451 - val_loss: 1.3975
Epoch 3/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 390s 167ms/step - loss: 1.3257 - val_loss: 1.3394
Epoch 4/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 390s 167ms/step - loss: 1.2343 - val_loss: 1.3010
Epoch 5/5
2338/2338 ━━━━━━━━━━━━━━━━━━━━ 390s 167ms/step - loss: 1.1478 - val_loss: 1.2769


In [16]:
# (1) sampling Search 기반 generate_text

def generate_text_sampling(model, tokenizer, init_sentence="<start>", max_len=30, temperature=1.0, top_k=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은  를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)

    for _ in range(max_len):
        preds = model(test_tensor)
        preds = preds[:, -1, :]  # 마지막 step

        # softmax
        prob = tf.nn.softmax(preds).numpy().flatten()

        # top-k 제한
        top_k_indices = prob.argsort()[-top_k:]
        top_k_probs = prob[top_k_indices]
        top_k_probs = top_k_probs / np.sum(top_k_probs)

        # sampling
        next_id = np.random.choice(top_k_indices, p=top_k_probs)

        # <end> 토큰이면 종료
        if tokenizer.index_word.get(next_id) == "<end>":
            break

        next_tensor = tf.convert_to_tensor([[next_id]], dtype=tf.int64)
        test_tensor = tf.concat([test_tensor, next_tensor], axis=-1)

    # 토큰 → 문장
    result = []
    for idx in test_tensor[0].numpy():
        w = tokenizer.index_word.get(idx, "")
        if w not in ["<start>", "<end>", "<pad>"]:
            result.append(w)

    return " ".join(result)

In [13]:
# (2) Beam Search 기반 generate_text

def generate_text_beam(model, tokenizer, init_sentence="<start>", max_len=30, beam_width=3):
    init_ids = tokenizer.texts_to_sequences([init_sentence])[0]

    sequences = [(init_ids, 0.0)]  # (sequence, score)

    for _ in range(max_len):
        candidates = []

        for seq, score in sequences:
            test_tensor = tf.convert_to_tensor([seq], dtype=tf.int64)

            preds = model(test_tensor)[:, -1, :]
            prob = tf.nn.softmax(preds).numpy().flatten()

            top_k = prob.argsort()[-beam_width:]

            for w in top_k:
                new_seq = seq + [w]
                new_score = score - np.log(prob[w] + 1e-8)

                candidates.append((new_seq, new_score))

        sequences = sorted(candidates, key=lambda x: x[1])[:beam_width]

    # 가장 좋은 sequence 선택
    best_seq = sequences[0][0]

    result = []
    for idx in best_seq:
        w = tokenizer.index_word.get(idx, "")
        if w not in ["<start>", "<end>", "<pad>"]:
            result.append(w)

    return " ".join(result)

In [36]:
print("=== Sampling ===")
generate_text_sampling(model, tokenizer, init_sentence="<start> it", temperature=0.8)

=== Sampling ===


"it wasn't <unk> , — this <unk>"

In [37]:
print("\n=== Beam Search ===")
generate_text_beam(model, tokenizer, init_sentence="<start> it", beam_width=3)


=== Beam Search ===


"it ain't too much stuff                         "